In [1]:
import os
import torch
from torch.utils.data import DataLoader, SequentialSampler

from datasets import SASRecDataset
from models import S3RecModel
from trainers import FinetuneTrainer

In [2]:
from utils import (
    check_path,
    generate_submission_file,
    get_item2attribute_json,
    get_user_seqs,
    set_seed,
    __save_labels
)

In [3]:
from collections import namedtuple
args = namedtuple("Config", [])

In [4]:
args.data_dir = "../../../data/train/"
args.output_dir = "output/"
args.data_name  = "rb"
args.model_name = "Finetune_full"
args.hidden_size = 64 # hidden size of transformer model
args.num_hidden_layers = 2 # number of layers
args.num_attention_heads = 2 # num_attention_heads
args.hidden_act = "gelu"
args.attention_probs_dropout_prob = 0.5 # attention dropout p
args.hidden_dropout_prob = 0.5 # hidden dropout p
args.initializer_range=0.02
args.max_seq_length=50

# train args
args.lr=0.001 # "learning rate of adam"
args.batch_size=256 # number of batch_size"
args.epochs=200#"number of epochs")
args.log_freq=1 #per epoch print res")
args.seed=42
args.weight_decay=0.0 #, help="weight_decay of adam"
args.adam_beta1=0.9 #"adam first beta value"
args.adam_beta2=0.999 #"adam second beta value"
args.gpu_id="0"#"gpu_id"

In [5]:
set_seed(args.seed)
check_path(args.output_dir) # output_dir 로 설정한 경로가 없으면 생성

In [6]:
args.no_cuda = False # gpu 있음
os.environ["CUDA_VISIBLE_DEVICES"] = args.gpu_id
args.cuda_condition = torch.cuda.is_available() and not args.no_cuda

In [7]:
args.data_file = args.data_dir + "train_ratings.csv"
item2attribute_file = args.data_dir + args.data_name + "_item2attributes.json"

In [8]:
import pandas as pd
# args.data_fil 여기경로를 지나쳐서 index에 관한 정보를 한 번 업데이트하고 지나가자.
# def get_user_seqs(data_file):
# 이거는 training 한정
rating_df = pd.read_csv(args.data_file)








# user_seq, max_item, _, _, submission_rating_matrix = get_user_seqs(args.data_file)
# item2attribute, attribute_size = get_item2attribute_json(item2attribute_file)

# args.item_size = max_item + 2
# args.mask_id = max_item + 1
# args.attribute_size = attribute_size + 1

In [ ]:
# le = LabelEncoder()
# if is_train:
#     # For UNKNOWN class
#     a = df[col].unique().tolist() + ["unknown"]
#     le.fit(a)
#     self.__save_labels(le, col)
# else:
#     label_path = os.path.join(self.args.asset_dir, col + "_classes.npy")
#     le.classes_ = np.load(label_path)

#     df[col] = np.where(df[col].isin(le.classes_), df[col], "unknown")


In [9]:
rating_df["item"].nunique()
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
raw_item_list = rating_df["item"].unique().tolist()
le.fit(raw_item_list)
__save_labels(le, "item")

TypeError: __save_labels() missing 1 required positional argument: 'name'

In [25]:
# save model args
args_str = f"{args.model_name}-{args.data_name}"

# print(args)

# args.item2attribute = item2attribute

# args.train_matrix = submission_rating_matrix

checkpoint = args_str + ".pt"
args.checkpoint_path = os.path.join(args.output_dir, checkpoint)

# submission_dataset = SASRecDataset(args, user_seq, data_type="submission")
# submission_sampler = SequentialSampler(submission_dataset)
# submission_dataloader = DataLoader(
#     submission_dataset, sampler=submission_sampler, batch_size=args.batch_size
# )

<class '__main__.Config'>


In [ ]:
# 따로 저장해야하는 것들
# max_item # 

In [28]:
model = S3RecModel(args=args)

AttributeError: type object 'Config' has no attribute 'item_size'